# Setting Up All Artifacts details

In [ ]:
## Give appropriate permission to the directory "FOLDER_WITH_ARTIFACTS" you are working with
import os
os.environ['SNPE_ROOT']="/local/mnt/workspace/qaisw-v2.15.1.230926150623_62883"#set up your snpe path here.
os.environ['RAW_FILE_FOLDER']="raw"#rawfiles
os.environ['FOLDER_WITH_ARTIFACTS']="StableLLVE"
os.environ['DLC32']="dlc/StableLLVE_fp32.dlc"
os.environ['DLC8']="dlc/StableLLVE_w8a8.dlc"
os.environ['TARGET_INPUT_LIST']="list.txt"
os.environ['ONDEVICE_FOLDER']="StableLLVE"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="2dce6316" #fill your device-id. Use command "adb devices" to get devices names. example :"e18d5d0"
os.environ['SNPE_TARGET_ARCH']="aarch64-android"
os.environ['SNPE_TARGET_STL']="libc++_shared.so"

In [ ]:
%%bash
git clone https://github.com/zkawfanx/StableLLVE.git

In [ ]:
%%bash
cp generate_model.py StableLLVE/
cd StableLLVE
python generate_model.py

#### Getting the Non Quantized DLC File

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-onnx-to-dlc -i StableLLVE.onnx -o dlc/StableLLVE_fp32.dlc

#### Understanding the Architecture of the Model

In [ ]:
%%bash 
source $SNPE_ROOT/bin/envsetup.sh
snpe-dlc-viewer -i dlc/StableLLVE_fp32.dlc -s StableLLVE.html
snpe-dlc-info -i dlc/StableLLVE_fp32.dlc>StableLLVE.txt

Use the dataset of your choice to validate the pre-processing, and post processing steps given in this notebook

In [ ]:
def preprocess(ll_img, hl_img):
    ll_img = np.array(ll_img, dtype='float32')
    hl_img = np.array(hl_img, dtype='float32')   
    ll_img = np.uint8(np.clip(ll_img, 0., 255.))  # this is to simulate matlab's imwrite operation
    hl_img = np.uint8(np.clip(hl_img, 0., 255.))  # this is to simulate matlab's imwrite operation 
    ll_img = np.uint8(ll_img)
    hl_img = np.uint8(hl_img)
    ll_img = convert_image(ll_img, source='array', target='[0, 1]')
    hl_img = convert_image(hl_img, source='array', target='[0, 1]')
    return ll_img, hl_img

## Preprocessing the data to generate raw file

In [ ]:
def convert_image(img, source, target):
    if source == 'array':
        img = torch.from_numpy(img.transpose((2, 0, 1))).contiguous()#chw
        img = img.to(dtype=torch.float32).div(255) 
    elif source == '[0, 1]':
        img = torch.clamp(img, 0, 1)  # useful to post-process output of models that can overspill
    
    if target == '[0, 1]':
        pass  # already in [0, 1]
    return img

In [ ]:
def post_process(img):
    img = img.detach().cpu().numpy()
    img = np.clip(255. * img, 0., 255.)
    img = np.uint8(img)
    img = img.transpose(1, 2, 0)#hwc
    return img

In [ ]:
def load_dataset(test_images_dir):
    # Input images for the model
    INPUTS_LL = []
    # Post-processed images for visualization
    IMAGES_LL = [] # LL:Low Light
    IMAGES_HL = [] # HL:High Light

    # Load the test images
    count=0
    for img_path in glob.glob(os.path.join(test_images_dir, '*')):
        l_img = cv2.resize(cv2.imread(img_path),[480,640],interpolation=cv2.INTER_CUBIC)
        l_img = cv2.cvtColor(l_img, cv2.COLOR_BGR2RGB)
        
        h_img = cv2.resize(cv2.imread(img_path.replace("low","high")),[480,640],interpolation=cv2.INTER_CUBIC)
        h_img = cv2.cvtColor(h_img, cv2.COLOR_BGR2RGB)
        
        ll_img, hl_img = preprocess(l_img, h_img)#chw

        INPUTS_LL.append(ll_img)#chw
        IMAGES_LL.append(post_process(ll_img))#hwc
        IMAGES_HL.append(post_process(hl_img))#hwc
    return INPUTS_LL, IMAGES_LL, IMAGES_HL

**Converting the low dataset to raw file format to give it to the model and do the inference**

In [ ]:
os.makedirs('raw', exist_ok=True)

In [ ]:
import glob
import os
import cv2
import numpy as np
import torch

test_images_dir = "eval15/low"
INPUTS_LL, IMAGES_LL, IMAGES_HL = load_dataset(test_images_dir)
print(len(INPUTS_LL),len(IMAGES_LL),len(IMAGES_HL))
for i, img_ll in enumerate(INPUTS_LL):
    img_ll = img_ll.cpu().detach().numpy()
    img_ll = img_ll.astype(np.float32)
    fid = open("raw/img_"+str(i)+ ".raw", 'wb')
    img_ll.tofile(fid)
    fid.close()

In [ ]:
def post_process_enhanced(img):
    img = np.fromfile(img, np.float32)
    img = img.reshape((3,640,480)).astype(np.float32)
    img = np.clip(img* 255. , 0., 255.)
    img = np.uint8(img)
    img = img.transpose(1, 2, 0)#hwc
    return img

#### Getting the Quantized Model

In [ ]:
total_iter = 15
print("Generating input_list \"small_raw_list.txt\" with {} iterations".format(total_iter))
with open("list.txt",'w') as f:
    for i in range(total_iter):
        f.write("raw/img_{}.raw\n".format(i))

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-dlc-quantize --input_dlc dlc/StableLLVE_fp32.dlc --input_list list.txt --use_enhanced_quantizer --use_adjusted_weights_quantizer --axis_quant --output_dlc dlc/StableLLVE_w8a8.dlc --enable_htp --htp_socs sm8550

## Model Inference

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/dsp/lib"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/$SNPE_TARGET_STL /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
$DEVICE_SHELL push $SNPE_ROOT/bin/$SNPE_TARGET_ARCH/snpe-net-run /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin
$DEVICE_SHELL push $SNPE_ROOT/lib/hexagon-v75/unsigned/*.so /data/local/tmp/snpeexample/dsp/lib
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/*.so /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/$ONDEVICE_FOLDER"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $DLC32 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $DLC8 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push raw /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER

#### Inferencing the Quantized Model

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_8b_DSP
export OUTPUT_DLC_QUANTIZED8=StableLLVE_w8a8.dlc
export ADSP_LIBRARY_PATH="/data/local/tmp/snpeexample/dsp/lib;/system/lib/rfsa/adsp;/system/vendor/lib/rfsa/adsp;/dsp"
export ONDEVICE_FOLDER="StableLLVE"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_QUANTIZED8 --input_list list.txt --output_dir $OUTPUT_FOLDER --use_dsp

#### Inferencing the Non-Quantized Model

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_32b_CPU
export OUTPUT_DLC_32=StableLLVE_fp32.dlc
export ONDEVICE_FOLDER="StableLLVE"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_32 --input_list list.txt  --output_dir $OUTPUT_FOLDER

In [ ]:
%%bash
rm -rf OUTPUT_8b_DSP
rm -rf OUTPUT_32b_CPU

#### Pulling the Output

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_8b_DSP OUTPUT_8b_DSP
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_32b_CPU OUTPUT_32b_CPU

### Post Processing and Calculating PSNR Value

In [ ]:
import math
def compute_psnr(img_pred, img_true, data_range=255., eps=1e-8):
    err = (img_pred - img_true) ** 2 
    err = np.mean(err)
    return 10. * math.log10((data_range ** 2) / (err + eps))

In [ ]:
def evaluate_average_psnr(el_images, hl_images): #(enhanced_light<model o/p>, high_light <groud_truth>)
    psnr = []
    for i in range(len(el_images)):
        el_img = cv2.imread(el_images[i], 1)
        hl_img = cv2.imread(hl_images[i], 1)
        psnr.append(compute_psnr(el_img,hl_img))
    average_psnr = np.mean(np.array(psnr))
    return average_psnr

In [ ]:
folder = ["OUTPUT_32b_CPU", "OUTPUT_8b_DSP"]
from PIL import Image
dict_folder_wise={}
for j in range(0,len(folder)):
    IMAGES_EL = []# EL: Enhanced Light (model output)
    File_LL=[]
    File_EL=[]
    File_HL=[]
    dict_folder_wise[str(folder[j])]=[]
    for i in range(0,15):
        # for prediction img
        IMAGES_EL.append(post_process_enhanced(folder[j]+"/Result_"+str(i)+"/363.raw"))  
        im = Image.fromarray(IMAGES_EL[i])
        im.save(folder[j]+"/Result_"+str(i)+"/prediction.jpeg")
        File_EL.append(folder[j]+"/Result_"+str(i)+"/prediction.jpeg")
        
        #for ground truth HL
        im = Image.fromarray(IMAGES_HL[i])
        im.save(folder[j]+"/Result_"+str(i)+"/groundtruth_HL.jpeg")
        File_HL.append(folder[j]+"/Result_"+str(i)+"/groundtruth_HL.jpeg")
        
        #for ground truth LL
        im = Image.fromarray(IMAGES_LL[i])
        im.save(folder[j]+"/Result_"+str(i)+"/groundtruth_LL.jpeg")
        File_LL.append(folder[j]+"/Result_"+str(i)+"/groundtruth_LL.jpeg")
        dict_folder_wise[str(folder[j])].append([File_LL[i],File_HL[i],File_EL[i]])
    print(folder[j]," :: ",evaluate_average_psnr(File_EL,File_HL))
    print("\n============================\n")

### Comaparison of Output between Non Quantized Model and Quantized Model

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread
import random as rand

def showImagesHorizontally(majors,index):
    fig, axs = plt.subplots(1, 4, figsize=(12,4))
    for nn, ax in enumerate(axs.flat):
        if nn<=2:
            column = majors[nn]
            column_rec_name = column.replace('\n', '_').replace(' ', '_')
            image = imread(majors[nn])
            ax.set_xlabel(majors[nn].split("/")[-1])
        else:
            image=imread(dict_folder_wise['OUTPUT_8b_DSP'][index][-1])
            ax.set_xlabel("Prediction 8b")
        ax.imshow(image)
        
    fig.suptitle("Comparison Between Ground Truth and Prediction", fontsize=16)
    plt.show()

for index in range(0,15):
    showImagesHorizontally(dict_folder_wise['OUTPUT_32b_CPU'][index],index)